# Profitable App Profiles for the App Store and Google Play Markets

This project shall analyse data from both the Google Play Store and the Apple App Store. The goal is to analyse which apps on either app store will likely generate the most profit. This shall be done on the assumption that the only revenue stream shall be from advertising revenue, therefore the revenue shall scale proportionally with the number of active users of each app.

As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play. The goal here is to use a sample set from each store to analyse at current trends in the respective mobile app stores in order to predict which future apps will be profitable and more likely to succeed. Apple Store data and documentation can be found [here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home), while Google Play data and documentation can be found [here](https://www.kaggle.com/lava18/google-play-store-apps/home).

In [1]:
#Explore Data Function
def explore_data(dataset, start, end, rows_and_columns=False):
    data_slice = dataset[start:end]
    for row in data_slice:
        print(row)
        print('\n') #Adds a new empty line after each row
        
    if rows_and_columns:
        print('Numbers of row', len(dataset))
        print('Numbers of columns', len(dataset[0]))

In [2]:
from csv import reader

#Apple App Store Data
open_file1 = open('my_datasets/Project1-Apps/AppleStore.csv')
read_file1 = reader(open_file1)
Apple_data = list(read_file1)
Apple_header = Apple_data[0]
Apple_data = Apple_data[1:]

#Google Play Store Data
open_file2 = open('my_datasets/Project1-Apps/googleplaystore.csv')
read_file2 = reader(open_file2)
Google_data = list(read_file2)
Google_header = Google_data[0]
Google_data = Google_data[1:]

In [3]:
print(Apple_header)
AppStore = explore_data(Apple_data,0,4,True)
print(Google_header)
PlayStore = explore_data(Google_data,0,4,True)

['', 'id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['1', '281656475', 'PAC-MAN Premium', '100788224', 'USD', '3.99', '21292', '26', '4', '4.5', '6.3.5', '4+', 'Games', '38', '5', '10', '1']


['2', '281796108', 'Evernote - stay organized', '158578688', 'USD', '0', '161065', '26', '4', '3.5', '8.2.2', '4+', 'Productivity', '37', '5', '23', '1']


['3', '281940292', 'WeatherBug - Local Weather, Radar, Maps, Alerts', '100524032', 'USD', '0', '188583', '2822', '3.5', '4.5', '5.0.0', '4+', 'Weather', '37', '5', '3', '1']


['4', '282614216', 'eBay: Best App to Buy, Sell, Save! Online Shopping', '128512000', 'USD', '0', '262241', '649', '4', '4.5', '5.10.0', '12+', 'Shopping', '37', '5', '9', '1']


Numbers of row 7197
Numbers of columns 17
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Pric

In [4]:
for row in Google_data:
    if len(row) != len(Google_header):
        i = Google_data.index(row)
        del Google_data[i]
        
print(len(Google_data))

for row in Apple_data:
    if len(row) != len(Apple_header):
        i = Apple_data.index(row)
        del Apple_data[i]
        
print(len(Apple_data))
    

10840
7197


Duplicate rows exist in the Google Play Store data (none in Apple App Store). The next task shall be to remove duplicates of each app, this shall be done by removing the versions with fewer reviews. This is based on the assumption that fewer reviews indicates an older version of the app data. In order to do this a  dictionary will be created where each dictionary key is a unique app name corresponding to value with the highest reviews. From this a new dataset will be created with only one entry per app.

In [5]:
duplicate_apps = []
unique_apps = []

for app in Google_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apps:', len(duplicate_apps))


Number of duplicate apps: 1181


In [6]:
reviews_max = {}
for app in Google_data:
    name = app[0]
    n_reviews = float(app[3])
    if (name in reviews_max) and (reviews_max[name] < n_reviews):
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

if len(reviews_max) != (len(Google_data) - len(duplicate_apps)):
    print('Error, lengths do not match!')
else:
    print('All good, lengths agree.')


All good, lengths agree.



Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:
 - We start by initializing two empty lists, android_clean and already_added.
 - We loop through the android data set, and for every iteration:
 - We isolate the name of the app and the number of reviews.
 - We add the current row (app) to the android_clean list, and the app name (name) to the already_cleaned list if:
    - The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and
    - The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for `reviews_max[name] == n_reviews`, we'll still end up with duplicate entries for some apps.

In [7]:
android_clean = []
already_added = []

for app in Google_data:
    name = app[0]
    n_reviews = float(app[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name) 
        
if len(android_clean) != (len(Google_data) - len(duplicate_apps)):
    print('Error, lengths do not match!')
else:
    print('All good, lengths agree.')


All good, lengths agree.


Now that we have removed duplicates the next task is to remove non-English language apps. We're not interested in keeping these apps, so we'll remove them. One way to go about this is to remove each app with a name containing a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;), and other symbols (+, *, /).

Behind the scenes, each character we use in a string has a corresponding number associated with it. For instance, the corresponding number for character 'a' is 97, character 'A' is 65, and character '爱' is 29,233. We can get the corresponding number of each character using the ord() built-in function.

The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

If an app name contains a character that is greater than 127, then it probably means that the app has a non-English name. Our app names, however, are stored as strings, so how could we take each individual character of a string and check its corresponding number?

In Python, strings are indexable and iterable, which means we can use indexing to select an individual character, and we can also iterate on the string using a for loop.

In order top fix this a function shall be defined to detect string values greater than 127.

In [8]:
def english_check(string):
    for character in string:
        if ord(character) > 127:
            return False
        
    return True

In [9]:
print(english_check('Instagram'))
print(english_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english_check('Docs To Go™ Free Office Suite'))
print(english_check('Instachat 😜'))

True
False
False
False


This is because emojis and characters like ™ fall outside the ASCII range and have corresponding numbers over 127. So may accidentally delete English apps if not careful. To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective. Will edit function.

In [10]:
def english_check(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    if non_ascii > 3:
        return False
    else:
        return True

In [11]:
print(english_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english_check('Docs To Go™ Free Office Suite'))
print(english_check('Instachat 😜'))

False
True
True


In [12]:
eng_G_app = []
for app in android_clean:
    if english_check(app[0]) == True:
        eng_G_app.append(app)
    
eng_ios_app = []
for app in Apple_data:
    if english_check(app[2]) == True:
        eng_ios_app.append(app)

In [13]:
print(len(eng_G_app))
print(len(eng_ios_app))

9614
6183


Only interested in apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps; we'll need to isolate only the free apps for our analysis

In [14]:
clean_ios = []
for app in eng_ios_app:
    price = float(app[5])
    if price == 0:
        clean_ios.append(app)
        
clean_G = []
for app in eng_G_app:
    if app[6] == 'Free':
        clean_G.append(app)

print('Length of final, cleaned, iOS list', len(clean_ios))
print('Length of final, cleaned, Play Store list', len(clean_G))

Length of final, cleaned, iOS list 3222
Length of final, cleaned, Play Store list 8863


# Analysing the Data

Aim is to determine the type of apps likely to attract more users because our revenue is highly influenced by the number of people using our apps.
To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we then develop it further.
3. If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.
<br>
Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.
Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.


In [15]:
#Will Generate a Frequency Table
def freq_table(dataset,index):
    table = {}
    for row in dataset:
        name = row[index]
        if name in table:
            table[name] += 1
        else:
            table[name] = 1
    for key in table:
        percentage = (table[key]/len(dataset))*100
        table[key] = percentage
        
    return table

In [16]:
#Will transform freq table into list of tuples and order them
def display_table(dataset,index):
    table = freq_table(dataset,index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
    
    table_sorted = sorted(table_display, reverse=True)
    for entry in table_sorted:
        print(entry[1],':', entry[0])

### Apple DataSet
Can see that Games are by far (58.16%) the most numerous free English app on the App store.
This is followed up by Entertainment (7.88%). Photo and Video consists of approx 5% of apps. All other apps each do not contribute more than 5%. 
<br>
Generally apps designed for fun or entertainment (games, entertainment, photos & videos etc.) rather than practical purposes tend to be more prevelant on the App Store. Apps designed for practical purposes (education, shopping, utilities etc.) make up only approximately 20% of all available apps. While entertainment apps are most numerous this does not indicate they have the most users.

In [17]:
#Apply to Apple Dataset
display_table(clean_ios, 12)


Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


### Google DataSet
Looking at the Catagory column first, it can be seen that Family is the most common app catagory in the Google Play Store with (18.9%). Further investigation into this catagory reveals it to consist primarily of kids games. The second most popular catagory is Game (9.7%). Considering these two catagories together, then games comprise 28.6% of games on the Play Store. This is considerably less than the 58% of the App Store.
</br>
Additionally, it appears that practical apps (Tools, Business, Lifestyle) are more numerous in the Play Store. This is futher emphasised when analysing the Genre column of the Play Store dataset, however Games appear less frequency according to this column.

In [18]:
#Apply to Google (Catagory) Dataset
display_table(clean_G, 1)


FAMILY : 18.898792733837304
GAME : 9.725826469592688
TOOLS : 8.462146000225657
BUSINESS : 4.592124562789123
LIFESTYLE : 3.9038700214374367
PRODUCTIVITY : 3.8925871601038025
FINANCE : 3.7007785174320205
MEDICAL : 3.5315355974275078
SPORTS : 3.396141261423897
PERSONALIZATION : 3.317161232088458
COMMUNICATION : 3.2381812027530184
HEALTH_AND_FITNESS : 3.0802211440821394
PHOTOGRAPHY : 2.944826808078529
NEWS_AND_MAGAZINES : 2.798149610741284
SOCIAL : 2.6627552747376737
TRAVEL_AND_LOCAL : 2.335552296062281
SHOPPING : 2.245289405393208
BOOKS_AND_REFERENCE : 2.1437436533904997
DATING : 1.8616721200496444
VIDEO_PLAYERS : 1.7939749520478394
MAPS_AND_NAVIGATION : 1.399074805370642
FOOD_AND_DRINK : 1.241114746699763
EDUCATION : 1.1621347173643235
ENTERTAINMENT : 0.9590432133589079
LIBRARIES_AND_DEMO : 0.9364774906916393
AUTO_AND_VEHICLES : 0.9251946293580051
HOUSE_AND_HOME : 0.8236488773552973
WEATHER : 0.8010831546880289
EVENTS : 0.7108202640189552
PARENTING : 0.6544059573507841
ART_AND_DESIGN : 0

In [19]:
#Apply to Google (Genres) Dataset
display_table(clean_G, 9)

Tools : 8.450863138892023
Entertainment : 6.070179397495204
Education : 5.348076272142616
Business : 4.592124562789123
Productivity : 3.8925871601038025
Lifestyle : 3.8925871601038025
Finance : 3.7007785174320205
Medical : 3.5315355974275078
Sports : 3.463838429425702
Personalization : 3.317161232088458
Communication : 3.2381812027530184
Action : 3.102786866749408
Health & Fitness : 3.0802211440821394
Photography : 2.944826808078529
News & Magazines : 2.798149610741284
Social : 2.6627552747376737
Travel & Local : 2.324269434728647
Shopping : 2.245289405393208
Books & Reference : 2.1437436533904997
Simulation : 2.042197901387792
Dating : 1.8616721200496444
Arcade : 1.8503892587160102
Video Players & Editors : 1.771409229380571
Casual : 1.7601263680469368
Maps & Navigation : 1.399074805370642
Food & Drink : 1.241114746699763
Puzzle : 1.128286133363421
Racing : 0.9928917973598104
Role Playing : 0.9364774906916393
Libraries & Demo : 0.9364774906916393
Auto & Vehicles : 0.9251946293580051
S

### Apple DataSet

Now want to find the genre with the most downloads. To do this, user rating shall be used as a proxy for determining the number of downloads of each app.

In [20]:
table = freq_table(clean_ios, 12)
for genre in table:
    total = 0
    len_genre = 0
    for app in clean_ios:
        genre_app = app[12]
        if genre_app == genre:
            total += float(app[6])
            len_genre += 1
    
    av_num_ratings = total / len_genre
    print(genre,':', av_num_ratings)

Productivity : 21028.410714285714
Weather : 52279.892857142855
Shopping : 26919.690476190477
Reference : 74942.11111111111
Finance : 31467.944444444445
Music : 57326.530303030304
Utilities : 18684.456790123455
Travel : 28243.8
Social Networking : 71548.34905660378
Sports : 23008.898550724636
Health & Fitness : 23298.015384615384
Games : 22788.6696905016
Food & Drink : 33333.92307692308
News : 21248.023255813954
Book : 39758.5
Photo & Video : 28441.54375
Entertainment : 14029.830708661417
Business : 7491.117647058823
Lifestyle : 16485.764705882353
Education : 7003.983050847458
Navigation : 86090.33333333333
Medical : 612.0
Catalogs : 4004.0


# Analysis of App Store Data

The most popular genre of app is Navigation Apps with 86090 ratings. However it can be seen that the apps such as Waze (67%) and Google Maps (30%) comprise the vast majority of these downloads. Second most popular is Reference apps with 74942 ratings. Similarly to the previous case this is dominated primarily by two apps with a high number of ratings which are the Bible and Dictionary.com. 
</br>
This trend follows through with Social Media apps, dominated by Facebook, Skype, Pinterest etc., and also with Music apps which are dominated by Pandora, Spotify, TuneIn Radio etc.
</br>
However from all these catagories the most promising for App development would be either a simple radio app in Music or a Reference app as there are only a couple apps in the market.

In [21]:
#Analyse the top apps for each catagory
for app in clean_ios:
    if app[12] == 'Navigation':
        print(app[2],':', app[6])
print('\n')

for app in clean_ios:
    if app[12] == 'Reference':
        print(app[2],':', app[6])
print('\n')
                
for app in clean_ios:
    if app[12] == 'Social Networking':
        print(app[2],':', app[6])
print('\n')

for app in clean_ios:
    if app[12] == 'Music':
        print(app[2],':', app[6])
print('\n')

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Geocaching® : 12811
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5
CoPilot GPS – Car Navigation & Offline Maps : 3582
Google Maps - Navigation & Transit : 154911


Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
Merriam-Webster Dictionary : 16849
Google Translate : 26786
Night Sky : 12122
WWDC : 762
Jishokun-Japanese English Dictionary & Translator : 0
教えて!goo : 0
VPN Express : 14
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
City Maps for Minecraft PE - The Best Maps for Minecraft Pock

# Most Popular Apps by Genre on Google Play
Will know look at the number of installs per app but must convert from stings to floats first.


In [22]:
categories_android = freq_table(clean_G, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in clean_G:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3697848.1731343283
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

Communications is the most genre with the most downloads. Lets investigate which apps are most popular.

In [23]:
for app in clean_G:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

It is clear that this category is dominated by a few incredibly popular apps such as WhatsApp, Skype, Gmail etc. Will no review the Books and Reference category which showed promise previously.

In [25]:
for app in clean_G:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

The book and reference genre includes a variety of apps: software for processing and reading ebooks, various collections of libraries, dictionaries, tutorials on programming or languages, etc. It seems there's still a small number of extremely popular apps that skew the average:

In [27]:
for app in clean_G:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Google Play Books : 1,000,000,000+
Bible : 100,000,000+
Amazon Kindle : 100,000,000+
Wattpad 📖 Free Books : 100,000,000+
Audiobooks from Audible : 100,000,000+


In [28]:
for app in clean_G:
    if app[1] == 'BOOKS_AND_REFERENCE' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
AlReader -any text book reader : 5,000,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
ReadEra – free ebook reader : 1,000,000+
Ebook Reader : 5,000,000+
Read books online : 5,000,000+
eBoox: book reader fb2 epub zip : 1,000,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
Moon+ Reader : 10,000,000+
English-Myanmar Dictionary : 1,000,000+
Golden Dictionary (EN-AR) : 1,000,000+
All Language Translator Free : 1,000,000+
Aldiko Book Reader : 10,000,000+
Dictionary - WordWeb : 5,000,000+
50000 Free eBooks & Free AudioBooks : 5,000,000+
Al-Quran (Free) : 10,000,000+
Al Quran Indonesia : 10,000,000+
Al'Quran Bahasa Indonesia : 10,000,000+
Al Quran Al karim : 1,000,000+
Al Quran : EAlim - Translations & MP3 Offline : 5,000,000+
Koran Read &MP3 30 Juz Offline : 1,000,000+
H

It is clear that there is a significant proportion of "middle of the road" apps which have recieved a significant number of downloads but is not one of the large market competitors. We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.